In [1]:
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import pickle
%xmode Verbose

Exception reporting mode: Verbose


In [2]:
def positional_encoding(vocab_len, d_model):
    j = np.arange(d_model)
    Encodings = np.zeros((vocab_len, d_model))
    for p in range(vocab_len):
        Encodings[p][0::2] +=  np.sin(p/np.power(10000,2*j[0::2]/d_model))
        Encodings[p][1::2] +=  np.cos(p/np.power(10000,2*j[0::2]/d_model))
    return Encodings

In [3]:
# """Uncomment to create padded sequence, it takes 12 mins"""

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences


# df_train = pd.read_csv('D:/Industry/Projects/Datasets/cnn_dailymail/train.csv')
# df_valid = pd.read_csv('D:/Industry/Projects/Datasets/cnn_dailymail/validation.csv')
# df = pd.concat([df_train, df_valid]).reset_index()

# import re
# def remove_special_words(text):
#     return re.sub(r'^@|^http|[^\w\s\U0001F600-\U0001F64F]','',text)

# df["article"] = df["article"].apply(remove_special_words).str.lower()
# df["highlights"] = df["highlights"].apply(remove_special_words).str.lower()
# tokenizer_input = Tokenizer()
# tokenizer_input.fit_on_texts(df['article'])
# sequences_input = tokenizer_input.texts_to_sequences(df['article'])
# padded_sequences_input = pad_sequences(sequences_input, padding='post')

# tokenizer_output = Tokenizer()
# tokenizer_output.fit_on_texts(df['highlights'])
# sequences_output = tokenizer_output.texts_to_sequences(df['highlights'])
# padded_sequences_output = pad_sequences(sequences_output, padding='post',maxlen = padded_sequences_input.shape[1])
# with open('sequences.pkl', 'wb') as file:
#     pickle.dump([padded_sequences_input,padded_sequences_output,tokenizer_input,tokenizer_output], file)

In [4]:
with open('sequences.pkl', 'rb') as file:
    sequences = pickle.load(file)

padded_sequences_input = sequences[0]
padded_sequences_output = sequences[1]
tokenizer_input = sequences[2]
tokenizer_output = sequences[3]

In [5]:
encoder_vocab = len(tokenizer_input.word_index)
decoder_vocab = len(tokenizer_output.word_index)

In [6]:
padded_sequences_input.shape,padded_sequences_output.shape

((300481, 2137), (300481, 2137))

In [9]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import backend as K
K.clear_session()
from models import Transformer
transformer_layer = Transformer(x_shape=(2137,),y_shape=(2137,),
                                num_heads = 10,
                                encoder_vocab = encoder_vocab, 
                                decoder_vocab = decoder_vocab,
                                max_len=2137)
model = transformer_layer.transformer()
print(model.summary(expand_nested=True,line_length = 110))
model.compile(optimizer='adam', 
              loss=SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
model.predict([padded_sequences_input[0:2],padded_sequences_output[0:2]]).shape

Model: "transformer_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape              ┃          Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ transformer_input_2            │ (None, 2137)              │                0 │ -                          │
│ (InputLayer)                   │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ transformer_input_1            │ (None, 2137)              │                0 │ -                          │
│ (InputLayer)                   │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ embedding_output (Embedding)   │ (None, 2137, 300)         │       71,751,600 │ transformer_input_2[0][0]  │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ embedding_input (Embedding)    │ (None, 2137, 300)         │      260,850,900 │ transformer_input_1[0][0]  │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ masked_attention (Functional)  │ (None, 2137, 300)         │        3,609,900 │ embedding_output[0][0]     │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│    └ masked_attention_input    │ (None, 2137, 300)         │                0 │ -                          │
│ (InputLayer)                   │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│    └ masked_attention          │ (None, 2137, 300)         │        3,609,300 │ -                          │
│ (MultiHeadAttention)           │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│    └ masked_attention_add      │ (None, 2137, 300)         │                0 │ -                          │
│ (Add)                          │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│    └                           │ (None, 2137, 300)         │              600 │ -                          │
│ masked_attention_layer_norm    │                           │                  │                            │
│ (LayerNormalization)           │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ encoder (Functional)           │ (None, 2137, 300)         │        4,211,800 │ embedding_input[0][0]      │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│    └ encoder_input             │ (None, 2137, 300)         │                0 │ -                          │
│ (InputLayer)                   │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│    └ encoder (Functional)      │ (None, 2137, 300)         │        3,609,900 │ -                          │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│       └ encoder_input          │ (None, 2137, 300)         │                0 │ -                          │
│ (I

 Total params: 344,726,300 (1.28 GB)

 Trainable params: 344,726,300 (1.28 GB)

 Non-trainable params: 0 (0.00 B)

None
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


(2, 2137, 300)

In [ ]:
padded_sequences_input[0:20].shape,padded_sequences_output[0:20].shape

((20, 2137), (20, 2137))

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

AttributeError: module 'pydot' has no attribute 'InvocationException'

In [ ]:
padded_sequences_input.shape

(300481, 2137)

In [ ]:
padded_sequences_output.shape

(300481, 2137)